**Lecture 2**

Decile Formation based on Book-to-market ratio, and Long Short Portfolio Returns calculation



**Data Description**

Important Dataframes

1.  "Returns" dataframe : It contains monthly returns(RET), shares  outstanding (SHROUT) values, Price (PRC), Primary Exchange Code (PRIMEXCH) and  Unique Identifiers (PERMNO). The data are downloaded from  CRSP.

Key Input data:
date:    yyyymmdd format
RET:     return for the month ending yyyymmdd
EXCHCD:  Exchange where listed
SHROUT:  Shares outstanding as of month ending yyyymmdd


2.   "Cstat" dataframe : It contains book value of common equity(ceq) values for firms with unique identifiers (PERMNO)



3. merged_data : Dataframe obtained from Merging "Returns" & "Cstat_book_eps" dataframe on "PERMNO" & "date" with 1 year tolerance for merging. Book to Market Ratio (b2m) is calculated using ceq and marketcap values.







In [1]:
# Importing Necessary Python Libraries
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
from pandas import DateOffset

CRSP Data

* date    : Month-end date
* PERMNO  : permanent identification number assigned by CRSP
* PRIMEXCH : Primary Exchange for the stock
* RET      : Monthly returns (as of Month-end date)
* PRC      : Month-end price  
*SHROUT    : number of shares outstanding as of month-end (in thousands)
*CFACSHR   : Cumulative factor to adjust shares outstanding for stock splits.
*For all stocks CFACSHR=1 as of the last day on the CRSP dataset. For example, AAPL last declared a split (a 4 for 1 stock split) and went *ex-split* during August, 2020 - ie an investor who owned 1  APPL share in July 2020, had 4 ex-split shares at the end of August. Therefore CFACSHR=1 from 8/31/2020 to 12/31/2022, and CFACSHR = 4 going back from 7/31/2020 to the prior ex-split date.
* PRIMEXCH   : Primary exchange of listing
* SHRCD      : Share code. 10, 11 and 12 denote ordinary common shares. other codes denote ADRS, ETFs, REITS, closed-end funds etc.





In [2]:
#CRSP Data

# Importing CRSP price and returns datasets
Returns = pd.read_csv("MonthlyRet_198001_202312csv.zip") #Importing Cleaned CRSP data

# Handling Missing values
Returns.PRC = abs(Returns.PRC)                                         # Converting Price Values to absolute numbers (CRSP sets PRC with a "-" symbol if it is comuted as bid-ask average when there is no actual trade)

# Market Cap Calculation
Returns['marketcap'] = Returns.SHROUT * Returns.PRC                    #  Market Capitalization as of month end
Returns['marketcap'] = Returns.groupby('PERMNO')['marketcap'].shift()  # Lagged Market Capitalization = market cap as of the end of the previous month
Returns['marketcap'] = np.where(Returns['marketcap'] < 10000, np.nan, Returns['marketcap']) # exclude marketcap < $10m

# Exchange Code Filters
exch_nyse_amex_Nasdaq = ['N', 'Q', 'A']
Returns = Returns[Returns.PRIMEXCH.isin(exch_nyse_amex_Nasdaq)].copy() #keeping only NYSE (N), AMEX(A) and Nasdaq (Q) stocks, ie. stocks listed on  US exchanges)

#Keep only ordinary common shares
ord_common_shares = [10, 11, 12]
Returns = Returns[Returns.SHRCD.isin(ord_common_shares)].copy()             #keeping only ordinary common shares - excludes unit trusts, ADRS, REITS, closed-end funds

# Minor Pre-processing
Returns.reset_index(inplace = True, drop = True)                                                # Reset Index

Returns = Returns[["PERMNO","PRIMEXCH","date","RET","PRC","SHROUT","marketcap"]].copy() # Reordering the columns for clarity
Returns.RET = pd.to_numeric(Returns.RET, errors = 'coerce')                      #RET denoted missing value with alphanumeric values. convert it to Numeric with the 'coerce' option to set nonnumeric value to nan.

Returns.dropna(inplace = True)
#CRSP Data , prepare Date-time for merging with Compustat data

Returns["date"] = pd.to_datetime(Returns["date"])                       # Convert  "date" to a DateTime object
Returns["year"] = Returns["date"].dt.year                              # Extracting year
Returns["month"] = Returns["date"].dt.month                            # Extracting month



In [3]:
permno_unique = Returns.PERMNO.unique()
len(permno_unique)

24257

Compustat data items


* GVKEY : Compustat stock id
* LPERMNO: CRSP Permno for the stock
* datadate: fiscal year-end date
*   ceq: Book value of common equity
*   epsfx: Earnings per share





In [4]:
#Compustat Data

# Importing Compustat Data

Cstat = pd.read_csv('Cstat_20250108.zip')     # Importing monthly Compustat Data
Cstat.rename(columns = {'LPERMNO' : 'PERMNO'}, inplace = True) # Renaming "LPERMNO" for merging Cstat_book_eps with Returns data
Cstat = Cstat[["PERMNO","datadate","ceq"]]        # Keeping only relevant columns for clarity
Cstat.reset_index(inplace = True, drop = True)         # Reset Index

# Datetime Manipulations
Cstat["datadate"] = pd.to_datetime(Cstat["datadate"])             # Convert to  DateTime object for datetime manipulations
Cstat['date'] = Cstat['datadate'].apply(lambda x: x + DateOffset(months=+5)) # Adding five months (using DataOffset library) assuming it takes at most 4 months (why is DateOffset = +5 ?) for the data to reach the market

Cstat.dropna(inplace = True)                                   # Dropping Missing Values


In [6]:
Cstat.head(30)

,PERMNO,datadate,ceq,date
0,54594,1991-05-31,193.778,1991-10-31
1,54594,1992-05-31,196.737,1992-10-31
2,54594,1993-05-31,189.216,1993-10-31
3,54594,1994-05-31,189.488,1994-10-31
4,54594,1995-05-31,197.119,1995-10-31
5,54594,1996-05-31,204.635,1996-10-31
6,54594,1997-05-31,269.259,1997-10-31
7,54594,1998-05-31,300.850,1998-10-31
8,54594,1999-05-31,326.035,1999-10-31
9,54594,2000-05-31,339.515,2000-10-31


Merge CRSP and Compusta data by PERMNO.

Ensure no look-ahead bias. Are the Compustat data available when CRSP price is repoted?

In [7]:
# Merged Data

Returns.sort_values(by = 'date', inplace = True)                       # Sort CRSP data by date to use merge_asof (Note: data should be sort on the variable that is used to "merge_asof")
Cstat.sort_values(by = 'date', inplace = True)                 # Sort Cstat data by date to use merge_asof


merged_data = pd.merge_asof(Returns, Cstat, by = 'PERMNO', left_on = 'date', right_on= 'date', tolerance=dt.timedelta(days = 365)) # Merging "Returns" & "Cstat_book_eps" dataframe on "PERMNO" & "date" with 1 year tolerance for date
merged_data.dropna(inplace = True)                                # Dropping Missing Values

# Calculating Book to Market Ratio
merged_data['b2m'] = merged_data.ceq / merged_data.marketcap      # Book to Market Ratio



print("***********************************************************")
print("merged_data Dataframe containing book-to-market ratio")
print("***********************************************************")
merged_data


***********************************************************
merged_data Dataframe containing book-to-market ratio
***********************************************************


,PERMNO,PRIMEXCH,date,RET,PRC,SHROUT,marketcap,year,month,datadate,ceq,b2m
555659,65816,A,1990-11-30,0.000000,2.87500,158992.0,4.571020e+05,1990,11,1990-06-30,414.900,0.000908
555669,85965,N,1990-11-30,0.141176,12.12500,12171.0,1.293169e+05,1990,11,1990-06-30,150.782,0.001166
555670,86765,A,1990-11-30,0.071429,1.87500,6024.0,1.054200e+04,1990,11,1990-06-30,-5.158,-0.000489
555679,59352,N,1990-11-30,0.022727,56.25000,18209.0,1.001495e+06,1990,11,1990-06-30,286.451,0.000286
555681,23035,Q,1990-11-30,0.041667,18.75000,4592.0,8.265600e+04,1990,11,1990-06-30,21.200,0.000256
...,...,...,...,...,...,...,...,...,...,...,...,...
2562981,21617,N,2023-12-29,0.115033,8.53000,501183.0,3.834050e+06,2023,12,2022-12-31,4439.000,0.001158
2562982,15395,N,2023-12-29,0.046065,556.59003,5617.0,2.988693e+06,2023,12,2022-12-31,1757.973,0.000588
2562984,91575,N,2023-12-29,0.209142,148.89000,28268.0,3.495621e+06,2023,12,2022-12-31,1108.700,0.000317
2562985,21612,N,2023-12-29,0.010292,23.56000,164286.0,3.831150e+06,2023,12,2022-12-31,1248.663,0.000326


In [8]:
len(merged_data)

1781196

In [10]:
# Compute decile portfolio returns
merged_data['rank'] = merged_data.groupby(['year','month'])['b2m'].transform(lambda x: pd.qcut(x, 10, duplicates='drop',labels=False)) # Calculating Ranks based on Book to Market Value in the Cross-section
merged_data = merged_data[merged_data.year >= 1990].copy()        # Including data with year greater than equal to 2000
merged_data.reset_index(inplace = True, drop = True)              # Reset Index

merged_data


,PERMNO,PRIMEXCH,date,RET,PRC,SHROUT,marketcap,year,month,datadate,ceq,b2m,rank
0,65816,A,1990-11-30,0.000000,2.87500,158992.0,4.571020e+05,1990,11,1990-06-30,414.900,0.000908,5
1,85965,N,1990-11-30,0.141176,12.12500,12171.0,1.293169e+05,1990,11,1990-06-30,150.782,0.001166,7
2,86765,A,1990-11-30,0.071429,1.87500,6024.0,1.054200e+04,1990,11,1990-06-30,-5.158,-0.000489,0
3,59352,N,1990-11-30,0.022727,56.25000,18209.0,1.001495e+06,1990,11,1990-06-30,286.451,0.000286,1
4,23035,Q,1990-11-30,0.041667,18.75000,4592.0,8.265600e+04,1990,11,1990-06-30,21.200,0.000256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781191,21617,N,2023-12-29,0.115033,8.53000,501183.0,3.834050e+06,2023,12,2022-12-31,4439.000,0.001158,7
1781192,15395,N,2023-12-29,0.046065,556.59003,5617.0,2.988693e+06,2023,12,2022-12-31,1757.973,0.000588,5
1781193,91575,N,2023-12-29,0.209142,148.89000,28268.0,3.495621e+06,2023,12,2022-12-31,1108.700,0.000317,3
1781194,21612,N,2023-12-29,0.010292,23.56000,164286.0,3.831150e+06,2023,12,2022-12-31,1248.663,0.000326,3


In [11]:
# Monthly Mean Portfolio Returns
meanret = merged_data.groupby(['year','month', 'rank'])['RET'].mean().to_frame()   # Calculating average return for each decile (according to b2m ratio) for each month
meanret

RET
year month rank          
1990 11    0    -0.044314
           1     0.100973
           2     0.076269
           3     0.084393
           4     0.069258
...                   ...
2023 12    5     0.112608
           6     0.127078
           7     0.129179
           8     0.153965
           9     0.168273

[3980 rows x 1 columns]

In [14]:
# Compute the difference between extreme portfolio returns and the Global mean
meanret = meanret.unstack(level = -1).copy()                                       # Unstacking the grouped dataframe
meanret[('RET', 'diff')] = meanret[('RET', 9)] -  meanret[('RET', 0)]              # Calculating the long short returns of the portfolio by substracting "rank 0" avg. return from "rank 9" avg. return

nmon = len(meanret)                                                                # nmon in number of months
#meanret = meanret.stack(level = -1).copy()                                         # Stacking the dataframe to year-month index level
meanret = meanret.stack(level=-1, future_stack=True).copy()
meanret.to_csv('teacherfile.csv') #output monthly returns

# Overall Portfolio Returns Statistics
global_mean = meanret.groupby('rank')['RET'].agg(['mean', 'std'])                # mean and standard deviation of mean
global_mean['t-stat'] =np.sqrt(nmon - 1) *  global_mean['mean']/global_mean['std'] # t-statistics calculation


global_mean

,mean,std,t-stat
rank,,,
0,0.006872,0.075273,1.819103
1,0.007841,0.064394,2.426166
2,0.009058,0.058663,3.076677
3,0.009127,0.054340,3.346669
4,0.009452,0.053604,3.513449
5,0.010823,0.053434,4.035743
6,0.010809,0.053051,4.059540
7,0.011991,0.053807,4.440276
8,0.013511,0.061096,4.406267
